In [8]:
#example:
#https://www.kaggle.com/code/purvitsharma/amazon-reviews-bidirectional-lstm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
#%matplotlib inline
plt.style.use("fivethirtyeight")
import seaborn as sns
sns.set_style("darkgrid")
#from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score
import re, string, nltk
#import emoji, bz2
#import emoji
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional, Dropout
from keras.layers import Embedding
#from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report
import warnings
warnings.filterwarnings("ignore")
import pickle

In [9]:
#might have to change this path depending on where you stored the datasets
product_training = pd.read_json("devided_dataset_v2/Grocery_and_Gourmet_Food/train/product_training.json")
review_training = pd.read_json("devided_dataset_v2/Grocery_and_Gourmet_Food/train/review_training.json")

product_training = product_training[0:5000]
review_training = review_training[0:5000]

#print(review_training['reviewText'])


In [10]:
#get rid of null values 
review_training1 = review_training[~review_training['reviewText'].isnull()]
print(review_training1)

                                  asin                        reviewerID  \
0     71F1F9B34E46A7F11E8B72C26CA861B6  D0A66D9D59DFB8808DE637FA50025550   
1     1755243A708579B595D1AAAFBADD7725  8D0A76D2383DCFC2B0DA3189FCC53490   
2     982EB9BA1A12C03C3635DCD2A6CFD79E  CEDFFFFA048A21A94B2272349949BD08   
3     08D86545D56A7C392B81B96F73DA85C0  8686C793EA21D71A574F18A847A733AC   
4     9A8BE2D7907DA443B9A72E712290C0B2  DE9971A7DF0055FCC6BF78AA387AEDA6   
...                                ...                               ...   
4995  135AE7982B3DA88FC2FFE2C2E01F459F  87987FC38EBC3AA041F73B07CF0B0619   
4996  2745501E4E72F413B5A4D57497FB103D  B9ECCF5C90192041883612EA8084C74E   
4997  544AD3792EAEDAB81BCC3852888229CC  89CCBF4312085987B3904C4CFA4EA661   
4998  E28B5A3AF6A4B81743EA7099B96FCE83  C698CC10E014E30617E9C2CE4E4C0EFD   
4999  F237E7AE3418C0DF1B715CCB16765203  C0DF2BE0169038C1333AC0D14A3CEC69   

      unixReviewTime  vote  verified   reviewTime  \
0         1438560000  None      Tr

In [11]:
def reshape_features(review_df, target_df):
        feature_df = pd.DataFrame(review_df, columns=['asin', 'reviewText'])
        # Merge data frames
        final_df = pd.merge(feature_df, target_df, on='asin')
        final_df = pd.DataFrame(final_df, columns=['reviewText', 'awesomeness'])
        return final_df

review_df = review_training1
target_df =  product_training[['asin','awesomeness']]
review_training1 = reshape_features(review_df, target_df)
print(review_training1)

                                            reviewText  awesomeness
0    Almost impossible to open, but the tea is firs...            0
1                                                 good            0
2    Arrived fresh. Delicious blend of spices.\nNON...            1
3                                  My favorite coffee.            1
4                           Just don't like the taste.            1
..                                                 ...          ...
651  My husband is a diabetic so I'm always on the ...            0
652  Bought to sample when the Stone Street Mayan O...            0
653  to every single pot of tea that I make I add t...            1
654  Probably one of the best coffees I have ever h...            0
655  Um... So I bought 16 whole *ounces*, huh? As i...            1

[656 rows x 2 columns]


In [12]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/anishafujii/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/anishafujii/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/anishafujii/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/anishafujii/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:

def clean_text(df, field):
    df[field] = df[field].str.replace(r"@"," at ")
    df[field] = df[field].str.replace("#[^a-zA-Z0-9_]+"," ")
    df[field] = df[field].str.replace(r"[^a-zA-Z(),\"'\n_]"," ")
    df[field] = df[field].str.replace(r"http\S+","")
    df[field] = df[field].str.lower()
    #print(df)
    return df

clean_text(review_training1,"reviewText")

,reviewText,awesomeness
0,"almost impossible to open, but the tea is firs...",0
1,good,0
2,arrived fresh delicious blend of spices \nnon...,1
3,my favorite coffee,1
4,just don't like the taste,1
...,...,...
651,my husband is a diabetic so i'm always on the ...,0
652,bought to sample when the stone street mayan o...,0
653,to every single pot of tea that i make i add t...,1
654,probably one of the best coffees i have ever h...,0


In [14]:
# Applying Lemmmatizer to remove tenses from texts.
# this is what takes a really long time when cleaning the entire dataset because of the for loop
# but just use recall_data() function to get already cleaned data for when running the full dataset 
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub('[^a-zA-Z0-9]',' ',text)
    #text= re.sub(emoji.get_emoji_regexp(),"",text)
    text = [lemmatizer.lemmatize(word) for word in text.split() if not word in set(stopwords.words('english'))]
    text = ' '.join(text)
    return text

review_training1["reviewTextClean"] = review_training1["reviewText"].apply(preprocess_text)
#print(review_training1.head(15))


In [15]:
review_training = review_training1[["awesomeness",'reviewTextClean']]
print(review_training)

     awesomeness                                    reviewTextClean
0              0              almost impossible open tea first rate
1              0                                               good
2              1  arrived fresh delicious blend spice non gmo pr...
3              1                                    favorite coffee
4              1                                         like taste
..           ...                                                ...
651            0  husband diabetic always lookout natural sweete...
652            0  bought sample stone street mayan organic decaf...
653            1  every single pot tea make add two finger pinch...
654            0         probably one best coffee ever great coffee
655            1  um bought whole ounce huh entire pound bag big...

[656 rows x 2 columns]


In [ ]:

# Stores products into a pickle file for easier training process
def store_data(data):
    # Store Product dict to speed up sentiment analysis for future runs
    with open('sentimentLSTM.pkl', 'wb') as fp:
        pickle.dump(data, fp)
        print('dictionary saved successfully to file')

store_data(review_training)

In [5]:
# Uses pickle file to recall features stored
def recall_data():
    # Recall stores sentiment analysis from file
    read_product = {}
    with open('sentimentLSTM.pkl', 'rb') as fp:
        read_product = pickle.load(fp)
        # print(read_product)
    return read_product


# if you run this line, don't need to load and clean all of the data 
review_training = recall_data()

In [16]:
X_train, X_test, y_train, y_test = train_test_split(np.array(review_training["reviewTextClean"]),np.array(review_training["awesomeness"]), test_size=0.25,random_state=42)
print(X_train.shape)
print(X_test.shape)


(492,)
(164,)


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf2 = TfidfVectorizer(use_idf=True, tokenizer=word_tokenize)
X_train_tf2 = tfidf2.fit_transform(X_train)
X_test_tf2 = tfidf2.transform(X_test)

In [18]:
#Deep Learning 
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split


X = review_training["reviewTextClean"]
y = review_training.awesomeness
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
print(X_train.shape)
print(X_test.shape)



(492,)
(164,)


In [19]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [20]:
# using tokenizer to transform text messages into training and testing set
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_seq_padded = pad_sequences(X_train_seq, maxlen=64)
X_test_seq_padded = pad_sequences(X_test_seq, maxlen=64)

X_train_seq_padded[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
       1136,  268,   32,  269,  103,   63,  359,  752,  571,  753,  443,
        572, 1137,  234,  270, 1138,  360, 1139,   64,   21], dtype=int32)

In [21]:
print(y_train.shape)
print(y_train[:10])
print(y_train.dtype)

y_train = y_train.astype(np.int32)
print(y_train.dtype)
print(y_train[:10])


(492,)
338    0
530    1
176    1
140    1
318    0
43     1
42     1
73     1
167    1
331    0
Name: awesomeness, dtype: int64
int64
int32
338    0
530    1
176    1
140    1
318    0
43     1
42     1
73     1
167    1
331    0
Name: awesomeness, dtype: int32


In [22]:
#this is where the code starts to differ a bit from the lstm model without hyperparameterization 
#this is also where I start to encounter errors 

#Deep Learning 
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils.vis_utils import plot_model
from keras.optimizers import Adam

def create_model(num_lstm_units, dropout_rate, recurrent_dropout_rate, embedding_dim):
    model = Sequential()
    model.add(Embedding(len(tokenizer.index_word)+1,embedding_dim))
    model.add(Bidirectional(LSTM(num_lstm_units, dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate)))
    model.add(Dense(128, activation="relu"))
    model.add(Dense(1,activation="sigmoid"))
    #optimizer = Adam(learning_rate=learning_rate)
    #model.compile(optimizer=optimizer,loss="binary_crossentropy",metrics=["accuracy"])
    model.compile("adam",loss="binary_crossentropy",metrics=["accuracy"])
    return model
#model.summary()
#plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)


# Define the hyperparameter search space
param_grid = {
    'num_lstm_units': [50, 100, 150],
    'dropout_rate': [0, 0.2, 0.4],
    'recurrent_dropout_rate': [0, 0.2, 0.4],
    'embedding_dim': [64, 128, 256],
    #'learning_rate': [0.001, 0.01, 0.1]
}


#Convert your Keras model into a scikit-learn compatible model
model = KerasClassifier(build_fn=create_model, epochs=15, batch_size=64, verbose=0)


# Perform the random search
random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=10, cv=3)
random_search.fit(X_train_seq_padded, y_train)

# Print the best hyperparameters and the corresponding accuracy
print("Best Hyperparameters: ", random_search.best_params_)
print("Best Accuracy: ", random_search.best_score_)

KeyboardInterrupt: 